# Advanced LBO Model Notebook

**Purpose:** This notebook provides a simplified, interactive model of a Leveraged Buyout (LBO). It demonstrates the core mechanics, including the sources & uses of funds, debt schedule, and returns calculation (IRR and MOIC). 

**Guide:** For a detailed explanation of the concepts, please refer to the [LBO Modeling Deep Dive guide](../../Financial_Modeling/Advanced_Topics/LBO_Modeling_Deep_Dive.md).

**How to Use:** Run the cells sequentially. Use the interactive sliders at the end to perform sensitivity analysis on key assumptions.

## 1. Setup and Library Imports

In [ ]:
import pandas as pd
import numpy as np
import numpy_financial as npf
from ipywidgets import interact, FloatSlider, IntSlider

pd.options.display.float_format = '{:,.2f}'.format

## 2. LBO Assumptions

We define all the key entry, financing, operational, and exit assumptions for our hypothetical LBO.

In [ ]:
# Entry Assumptions
entry_ebitda = 100.0
entry_multiple = 10.0
cash_on_bs = 20.0
existing_debt = 150.0
transaction_fees = 25.0

# Financing Assumptions
tla_multiple = 3.0 # Term Loan A as x of EBITDA
tla_interest_rate = 0.06
tla_amort_percent = 0.10 # 10% mandatory amortization per year

tlb_multiple = 3.0 # Term Loan B as x of EBITDA
tlb_interest_rate = 0.08
tlb_amort_percent = 0.01 # 1% mandatory amortization per year

# Operational Projections
proj_years = 5
revenue_growth = 0.05
ebitda_margin = 0.40 # Assuming constant for simplicity
tax_rate = 0.25
capex_percent_of_rev = 0.05
nwc_percent_of_rev = 0.10

# Exit Assumption
exit_multiple = 11.0

## 3. Transaction Calculation: Sources & Uses

Here, we calculate the total funding required (Uses) and how it's paid for (Sources). The Sponsor Equity is the plug that makes the two sides equal.

In [ ]:
# --- Uses ---
entry_ev = entry_ebitda * entry_multiple
equity_purchase_price = entry_ev - existing_debt + cash_on_bs

total_uses = equity_purchase_price + existing_debt + transaction_fees

# --- Sources ---
tla_debt = entry_ebitda * tla_multiple
tlb_debt = entry_ebitda * tlb_multiple
total_debt = tla_debt + tlb_debt

sponsor_equity = total_uses - total_debt

# --- Build S&U Table ---
sources = {'Term Loan A': tla_debt, 'Term Loan B': tlb_debt, 'Sponsor Equity': sponsor_equity}
uses = {'Equity Purchase Price': equity_purchase_price, 'Repay Existing Debt': existing_debt, 'Transaction Fees': transaction_fees}

su_df = pd.DataFrame({'Sources': pd.Series(sources), 'Uses': pd.Series(uses)}).fillna(0)
su_df.loc['Total'] = [su_df['Sources'].sum(), su_df['Uses'].sum()]

print("--- Sources & Uses ---")
display(su_df)

## 4. Projections & Debt Schedule

This is the core of the model. We project the company's cash flows and use them to pay down debt in a 'cash sweep'.

In [ ]:
def run_lbo_model(initial_tla, initial_tlb):
    # --- Initialize DataFrames ---
    years = np.arange(1, proj_years + 1)
    pnl = pd.DataFrame(index=['Revenue', 'EBITDA', 'D&A', 'EBIT', 'Interest Expense', 'EBT', 'Taxes', 'Net Income'], columns=years)
    cfs = pd.DataFrame(index=['Net Income', 'D&A', 'Change in NWC', 'CFO', 'CapEx', 'FCF', 'Mandatory Debt Paydown', 'Cash Sweep', 'Net Debt Change', 'Levered FCF'], columns=years)
    debt_schedule = pd.DataFrame(index=['TLA_Beg', 'TLA_Mand_Amort', 'TLA_Sweep', 'TLA_End', 'TLB_Beg', 'TLB_Mand_Amort', 'TLB_Sweep', 'TLB_End'], columns=years)
    
    # --- Set Initial Values ---
    revenue_t0 = entry_ebitda / ebitda_margin
    nwc_t0 = revenue_t0 * nwc_percent_of_rev
    debt_schedule.loc[['TLA_Beg', 'TLB_Beg'], 1] = [initial_tla, initial_tlb]
    
    # --- Loop Through Projections ---
    for year in years:
        # P&L Projections
        pnl.loc['Revenue', year] = revenue_t0 * (1 + revenue_growth)
        pnl.loc['EBITDA', year] = pnl.loc['Revenue', year] * ebitda_margin
        pnl.loc['D&A', year] = pnl.loc['Revenue', year] * capex_percent_of_rev # Simple assumption D&A = CapEx
        pnl.loc['EBIT', year] = pnl.loc['EBITDA', year] - pnl.loc['D&A', year]
        
        # Debt & Interest
        if year > 1:
            debt_schedule.loc['TLA_Beg', year] = debt_schedule.loc['TLA_End', year - 1]
            debt_schedule.loc['TLB_Beg', year] = debt_schedule.loc['TLB_End', year - 1]
        
        interest_tla = debt_schedule.loc['TLA_Beg', year] * tla_interest_rate
        interest_tlb = debt_schedule.loc['TLB_Beg', year] * tlb_interest_rate
        pnl.loc['Interest Expense', year] = interest_tla + interest_tlb
        
        pnl.loc['EBT', year] = pnl.loc['EBIT', year] - pnl.loc['Interest Expense', year]
        pnl.loc['Taxes', year] = -pnl.loc['EBT', year] * tax_rate
        pnl.loc['Net Income', year] = pnl.loc['EBT', year] + pnl.loc['Taxes', year]
        
        # Cash Flow Statement
        cfs.loc['Net Income', year] = pnl.loc['Net Income', year]
        cfs.loc['D&A', year] = pnl.loc['D&A', year]
        nwc_t1 = pnl.loc['Revenue', year] * nwc_percent_of_rev
        cfs.loc['Change in NWC', year] = -(nwc_t1 - nwc_t0)
        cfs.loc['CFO', year] = cfs.loc[['Net Income', 'D&A', 'Change in NWC'], year].sum()
        cfs.loc['CapEx', year] = -pnl.loc['Revenue', year] * capex_percent_of_rev
        cfs.loc['FCF', year] = cfs.loc['CFO', year] + cfs.loc['CapEx', year]
        
        # Debt Paydown (Cash Sweep Waterfall)
        cash_available_for_sweep = cfs.loc['FCF', year]
        
        debt_schedule.loc['TLA_Mand_Amort', year] = -min(initial_tla * tla_amort_percent, debt_schedule.loc['TLA_Beg', year])
        cash_available_for_sweep += debt_schedule.loc['TLA_Mand_Amort', year]
        
        debt_schedule.loc['TLB_Mand_Amort', year] = -min(initial_tlb * tlb_amort_percent, debt_schedule.loc['TLB_Beg', year])
        cash_available_for_sweep += debt_schedule.loc['TLB_Mand_Amort', year]
        
        tla_sweep = -min(max(0, cash_available_for_sweep), debt_schedule.loc['TLA_Beg', year] + debt_schedule.loc['TLA_Mand_Amort', year])
        debt_schedule.loc['TLA_Sweep', year] = tla_sweep
        cash_available_for_sweep += tla_sweep
        
        tlb_sweep = -min(max(0, cash_available_for_sweep), debt_schedule.loc['TLB_Beg', year] + debt_schedule.loc['TLB_Mand_Amort', year])
        debt_schedule.loc['TLB_Sweep', year] = tlb_sweep
        
        debt_schedule.loc['TLA_End', year] = debt_schedule.loc['TLA_Beg', year] + debt_schedule.loc['TLA_Mand_Amort', year] + debt_schedule.loc['TLA_Sweep', year]
        debt_schedule.loc['TLB_End', year] = debt_schedule.loc['TLB_Beg', year] + debt_schedule.loc['TLB_Mand_Amort', year] + debt_schedule.loc['TLB_Sweep', year]
        
        # Update CFS
        cfs.loc['Mandatory Debt Paydown', year] = debt_schedule.loc['TLA_Mand_Amort', year] + debt_schedule.loc['TLB_Mand_Amort', year]
        cfs.loc['Cash Sweep', year] = debt_schedule.loc['TLA_Sweep', year] + debt_schedule.loc['TLB_Sweep', year]
        cfs.loc['Net Debt Change', year] = cfs.loc[['Mandatory Debt Paydown', 'Cash Sweep'], year].sum()
        cfs.loc['Levered FCF', year] = cfs.loc['FCF', year] + cfs.loc['Net Debt Change', year]
        
        # Update for next loop
        revenue_t0 = pnl.loc['Revenue', year]
        nwc_t0 = nwc_t1
        
    return pnl, cfs, debt_schedule

pnl_df, cfs_df, debt_df = run_lbo_model(tla_debt, tlb_debt)

print("--- Income Statement ---")
display(pnl_df)
print("\n--- Cash Flow Statement ---")
display(cfs_df)
print("\n--- Debt Schedule ---")
display(debt_df)


## 5. Exit & Returns Calculation

Finally, we calculate the exit value and the returns to the sponsor.

In [ ]:
def calculate_returns(pnl, debt_schedule, sponsor_equity_investment, exit_multiple_val):
    # Exit Calculation
    exit_year = pnl.columns[-1]
    exit_ebitda = pnl.loc['EBITDA', exit_year]
    exit_ev = exit_ebitda * exit_multiple_val
    
    debt_at_exit = debt_schedule.loc['TLA_End', exit_year] + debt_schedule.loc['TLB_End', exit_year]
    exit_equity_value = exit_ev - debt_at_exit
    
    # Returns Calculation
    moic = exit_equity_value / sponsor_equity_investment
    
    # IRR Calculation
    cash_flows = [-sponsor_equity_investment] + [0] * (proj_years - 1) + [exit_equity_value]
    irr = npf.irr(cash_flows)
    
    # Display Results
    print(f"--- Exit & Returns Analysis (Exit Multiple: {exit_multiple_val:.1f}x) ---")
    print(f"Exit Year EBITDA:       {exit_ebitda:,.2f}")
    print(f"Exit Enterprise Value:  {exit_ev:,.2f}")
    print(f"Debt at Exit:           ({-debt_at_exit:,.2f})")
    print("-----------------------------------------")
    print(f"Exit Equity Proceeds:   {exit_equity_value:,.2f}")
    print(f"Initial Sponsor Equity: {sponsor_equity_investment:,.2f}")
    print("-----------------------------------------")
    print(f"MOIC:                   {moic:.2f}x")
    print(f"IRR:                    {irr:.2%}")

# Main calculation with interactive sliders
@interact(exit_mult=FloatSlider(min=8.0, max=14.0, step=0.5, value=exit_multiple, description='Exit Multiple'))
def interactive_returns(exit_mult):
    calculate_returns(pnl_df, debt_df, sponsor_equity, exit_mult)
